# Imports

In [5]:
import ipywidgets as widgets
import numpy as np
from IPython.display import display
import requests
import urllib.request
import json
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from bs4 import BeautifulSoup
from abc import ABC, abstractmethod
import aiohttp
import asyncio
from aiohttp import ClientSession



In [52]:
class Parser(ABC):

    is_json = False

    @abstractmethod
    def get_price(self, response_text, card_name):
        pass

class HobbymasterParser(Parser):
    name = 'Hobbymaster'
    url = 'https://hobbymaster.co.nz/cards/get-cards?foil=0&lang=&game=1&_search=true&sidx=set&sord=desc&name='
    is_json = True

    def get_price(self, response_text, card_name):
        data = response_text
        price_list = []
        if 'rows' in data:
            for item in data['rows']:
                if card_name.lower() in item['cell'][0].lower():
                    if item['cell'][12] == 0:
                        continue
                    else:
                        price = item['cell'][10]
                        price = float(price.replace('$', '').replace('!', ''))
                        price_list.append(price)
            if len(price_list) == 0:
                price = np.nan
            else:
                price = min(price_list)
        else:
            price = np.nan
        return price

class BaydragonParser(Parser):
    name = 'Baydragon'
    url = 'https://www.baydragon.co.nz/search/category/01?searchType=single&searchString='

    def get_price(self, response_text, card_name):
        soup = BeautifulSoup(response_text)
        div = soup.find('div', attrs={'class': 'tcgSingles'})
        table = div.find('table')
        tds = table.find_all('td')
        n = 0
        price_list = []
        for td in tds:
            n += 1
            if n > len(tds):
                break
            if 'NZ$' in td.text:
                if card_name in tds[n-6].text:
                    if int(tds[n].text) > 0:
                        price = float(td.text.replace('NZ$', ''))
                        price_list.append(price)
        if len(price_list) == 0:
            price = np.nan
        else:
            price = min(price_list)
        return price

class GoblinGamesParser(Parser):
    name = 'Goblin Games'
    url = 'https://goblingames.nz/search?q='

    def get_price(self, response_text, card_name):
        soup = BeautifulSoup(response_text)
        products = soup.find_all('div', attrs={'class': 'productCard__lower'})
        price_list = []
        for product in products:
            p_list = product.find_all('ul')
            for item in p_list:
                instock = item.find_all('li')
                if len(instock):
                    for i in instock:
                        title = i.attrs['data-producttitle']
                        price = int(i.attrs['data-price']) / 100
                    if card_name.lower() in title.lower():
                        price_list.append(price)
        if len(price_list) == 0:
            price = np.nan
        else:
            price = min(price_list)
        return price

class ShopifyParser(Parser):
    def __init__(self, url, name):
        self.url = url
        self.name = name
    def get_price(self, response_text, card_name):
    #     soup = BeautifulSoup(html_content, "lxml")
        soup = BeautifulSoup(response_text)
        products = soup.find_all('div', attrs={'class': 'product Norm'})
        price_list = []
        for product in products:
            title = product.find('p', attrs='productTitle').text.replace('\n', ' ').strip()
            price = product.find('p', attrs='productPrice').text.replace('\n', ' ').strip()
            if card_name.lower() in title.lower():
                if price == 'Varies':
                    prices = product.parent.find('div', attrs={'class': 'buyWrapper'}).find_all('p')
                    for p in prices:
                        try:
                            price = float(p.text.split('$')[1])
                        except:
                            continue
                        price_list.append(price)
                elif '$' in price:
                    try:
                        price = float(price.replace('$', ''))
                    except:
                        continue
                    price_list.append(price)
                else:
                    if price != 'Sold Out':
                        price_list.append(price)
        if len(price_list) == 0:
            price = np.nan
        else:
            price = min(price_list)

        return price



class CardSearcher():
    df_data = []
    cards = []
    def __init__(self, parsers):
        self.parsers = parsers

    async def run_search(self, cards):
        self.cards + cards
        await card_searcher.find_cards(cards)
        return display_prices

    async def get_prices(self, card):
        prices = [card]
        async with aiohttp.ClientSession() as session:
            for parser in self.parsers:
                async with session.get(parser.url + card) as resp:
                    response = await resp.json() if parser.is_json else await resp.text()
                    prices.append(parser.get_price(response, card))
        return prices
                    
    async def find_cards(self, cards):
        for card in cards:
            price = await self.get_prices(card)
            self.df_data.append(price)

    def display_prices(self):
        # global results_df
        headers = ["Card"] + list(parser.name for parser in self.parsers)
        results_df = pd.DataFrame(self.df_data, columns =headers)
        return results_df
        

In [58]:
parsers = [
    # HobbymasterParser(),
    BaydragonParser(),
    GoblinGamesParser(),
    ShopifyParser('https://spellboundgames.co.nz/search?q=', 'Spellbound'),
    ShopifyParser('https://magicatwillis.co.nz/search?q=', 'Magic at Willis'),
    ShopifyParser('https://ironknightgaming.co.nz/search?q=', 'Iron Knight Gaming'),
    ShopifyParser('https://mtgmagpie.com/search?q=', 'Magic Magpie')
]
        
def highlight_min(data, color='green'):
    '''
    highlight the maximum in a Series or DataFrame
    '''
    attr = 'background-color: {}'.format(color)
    if data.ndim == 1:  # Series from .apply(axis=0) or axis=1
        is_min = data == data.min()
        return [attr if v else '' for v in is_min]
    else:  # from .apply(axis=None)
        is_min = data == data.min().min()
        return pd.DataFrame(np.where(is_min, attr, ''),
                            index=data.index, columns=data.columns)

def start_card_search(button):
    cards = card_input.value.split('\n')
    card_searcher = CardSearcher(parsers)
    out = widgets.Output()
    with out:
        display(card_searcher.run_search(cards))
    # # with results_output:
    # #     display(widgets.HTML(f"Total cost: <b>NZ${sum(total_value)}</b> {cards_not_found}"))
    # with table_output:
    #     display(results_df.style.apply(highlight_min, axis=1, subset=list(parser.name for parser in card_searcher.parsers)))

card_input = widgets.Textarea(placeholder='Enter a cards on new lines', rows=5)
go_button = widgets.Button(description='Find prices', button_style='')
results_output = widgets.Output()
table_output = widgets.Output()
display(card_input, go_button)
cards = card_input.value.split('\n')
go_button.on_click(start_card_search)

Textarea(value='', placeholder='Enter a cards on new lines', rows=5)

Button(description='Find prices', style=ButtonStyle())

In [48]:


# global results_df
# chunks = [prices[x:x+len(parsers)] for x in range(0, len(cards), len(parsers))]
# headers = list(p.name for p in parsers)
# results_df = pd.DataFrame(chunks, columns =headers)
# results_df['Card'] = cards
# results_df
    

,Card,Baydragon,Goblin Games,Spellbound,Magic at Willis,Iron Knight Gaming,Magic Magpie
0,Prismari Campus,NaN,NaN,NaN,NaN,NaN,NaN
1,Dimir Aqueduct,0.57,0.5,NaN,NaN,0.4,0.50
2,Crosis's Catacomb,NaN,2.0,NaN,NaN,2.8,1.75


In [ ]:
cards = '''the ozolith
'''


In [ ]:

results_df

In [ ]:
df